In [58]:
import scipy, numpy as np
from scipy.stats import norm
from numpy import exp, sqrt, expm1, log
from enum import Enum

In [59]:
class OptType(Enum):
    CALL=1
    PUT=2

CALL = OptType.CALL
PUT = OptType.PUT

def G(K, F):
    return log(K/F)
def Ginv(z, F):
    return F*exp(z)
def BS(opt_type, F, K, vol, T):
    assert opt_type in [OptType.CALL, OptType.PUT]
    stddev = vol * sqrt(T)
    d1 = log(F/K)/stddev + 0.5*stddev
    d2 = log(F/K)/stddev - 0.5*stddev
    if opt_type == CALL:
        return F*norm.cdf(d1) - K*norm.cdf(d2)
    else:
        return K*norm.cdf(-d2) - F*norm.cdf(-d1)

def H_opt(opt_type, vol, T, alpha, L):
    assert opt_type in [OptType.CALL, OptType.PUT]
    stddev = vol * sqrt(T)
    d = (0.5*(alpha - 1.0)*stddev*stddev - L)/stddev
    d1 = d + 0.5*alpha * stddev
    d2 = d - 0.5*alpha * stddev

    cd1  = norm.cdf(d1)
    cd2  = norm.cdf(d2)
    cmd1 = norm.cdf(-d1)
    cmd2 = norm.cdf(-d2)

    if opt_type == PUT:
        #print(cmd2)
        C = stddev*(cmd2 - cmd1)/(alpha*stddev)     
    else:
        #print(cd1)
        C = stddev*(cd1 - cd2)/(alpha*stddev)

    e1 = expm1(alpha*stddev*d)
    e = e1 + 1.0
    if opt_type == OptType.CALL:
        C += e1 / alpha * cd1
    else:
        C -= e1 / alpha * cmd1

    if opt_type == CALL:
        nd1 = norm.pdf(d1)
        C1= - e * cd1
        C2= e*(alpha * cd1 + 1.0 / stddev * nd1)
        C3= -e/(stddev*stddev)*(alpha*alpha*stddev*stddev*cd1-nd1*(d-1.5*alpha*stddev))
    else:
        nd1 = norm.pdf(d1)
        C1= e* cmd1
        C2 = -alpha * e*cmd1 + e / stddev * norm.pdf(d1)
        C3= e/(stddev*stddev)*(alpha*alpha*stddev*stddev*cmd1+nd1*(d-1.5*alpha*stddev))
    return C, C1, C2,C3

In [60]:
vol = 0.20
T   = 5.0
alpha = 1.0
F   = 0.02
K   = 0.04

In [61]:
# Some simple sanity checks
alpha = 1.0
for K in [0.01,0.02,0.03]:
    for w in [CALL,PUT]:
        fv1 = K*H_opt(w,vol,T,alpha,G(K, F))[0]
        fv2 = BS(w,F,K,vol,T)
        print('fv1={} fv2={}'.format(fv1,fv2))
        assert(abs(fv1 - fv2)<1.0e-8)
        
alpha = 0.75
for K in [0.01,0.02,0.03]:
    for w in [CALL,PUT]:
        _,dfv1,ddfv1,dddfv1 = H_opt(w,vol,T,alpha,G(K, F))
        dfv2 = (H_opt(w,vol,T,alpha,G(K, F)+0.0001)[0]-H_opt(w,vol,T,alpha,G(K, F)-0.0001)[0])/0.0002
        ddfv2 = (H_opt(w,vol,T,alpha,G(K, F)+0.001)[0]- 2*H_opt(w,vol,T,alpha,G(K, F))[0] +H_opt(w,vol,T,alpha,G(K, F)-0.001)[0])/(0.001**2)
        dddfv2=(H_opt(w,vol,T,alpha,G(K, F)+0.0001)[2]-H_opt(w,vol,T,alpha,G(K, F)-0.0001)[2])/0.0002
        print('dfv1={} dfv2={}'.format(dfv1,dfv2))
        print('ddfv1={} ddfv2={}'.format(ddfv1,ddfv2))
        print('dddfv1={} dddfv2={}'.format(dddfv1,dddfv2))

fv1=0.010162265323490585 fv2=0.010162265323490587
fv1=0.00016226532349058618 fv2=0.00016226532349058608
fv1=0.003538734524837573 fv2=0.003538734524837571
fv1=0.003538734524837573 fv2=0.003538734524837571
fv1=0.0010702749421714198 fv2=0.001070274942171418
fv1=0.011070274942171419 fv2=0.011070274942171417
dfv1=-1.5708581958734191 dfv2=-1.5708581959772427
ddfv1=1.5483189766142353 ddfv2=1.5483190982656936
dddfv1=-0.06339696681146197 dddfv2=-0.06339697997792548
dfv1=0.07969480766378466 dfv2=0.07969480910648141
ddfv1=0.31040422396133255 ddfv2=0.31040428720430224
dddfv1=0.8650390976782152 dddfv2=0.8650390853806589
dfv1=-0.5343959038993072 dfv2=-0.534395906212487
ddfv1=1.2708338953107732 ddfv2=1.2708336376010543
dddfv1=-1.3881439051762268 dddfv2=-1.388143890617144
dfv1=0.4470287838484699 dfv2=0.4470287824551167
ddfv1=0.5347653794999405 ddfv2=0.5347650872955523
dddfv1=-0.836092518318102 dddfv2=-0.8360925032419031
dfv1=-0.154484682145218 dfv2=-0.15448468486223277
ddfv1=0.5868353321409305 ddfv2=0

In [62]:
from scipy.integrate import fixed_quad
N=10
def V_opt(opt_type, F, K, vol, T, alpha, mu):
    H,HL,HLL,HLLL = H_opt(opt_type, vol, T, alpha, G(K, F) - mu)
    
    V = K * H
    Vmu = -K * HL
    Vmu2 = K * HLL
    Vmu3 = -K * HLLL

    if opt_type == CALL:
        Kmax=1.0
        #V+= (1.0 - alpha)*fixed_quad(lambda k: H_opt(opt_type, vol, T, alpha, G(k, F)), K, 1., n=N)[0]
        res = (1.0 - alpha)*fixed_quad(lambda z: Ginv(z, F)*H_opt(opt_type, vol, T, alpha, z - mu), G(K,F), G(Kmax, F), n=N)[0]
        V   += res[0]
        Vmu -= res[1]
        Vmu2 += res[2]
        Vmu3 -= res[3]
    else:
        res = (1.0 - alpha)*fixed_quad(lambda k: H_opt(opt_type, vol, T, alpha, G(k, F) - mu), 0., K, n=N)[0]
        V    -= res[0]
        Vmu  += res[1]
        Vmu2 -= res[2]
        Vmu3 += res[3]
    #V += fixed_quad(lambda k: 1.0, a=K, b=1.0, n=N)[0]
    return V, Vmu, Vmu2, Vmu3

In [63]:
K=0.02
alpha=0.5
for w in [CALL, PUT]:
    fv1,fv1_mu, fv1_mu2, fv1_mu3=V_opt(w, F, K, vol, T, alpha, mu=0.0)
    fv2=BS(w,F,K,vol,T)
    print('fv1={} fv2={} diff={}'.format(fv1, fv2, fv1-fv2))
    fv1_mu_ = (V_opt(w, F, K, vol, T, alpha, mu=0.001)[0] - V_opt(w, F, K, vol, T, alpha, mu=-0.001)[0])/0.002
    print('fv1_mu={} fv1_mu_={} diff={}'.format(fv1_mu, fv1_mu_, fv1_mu - fv1_mu_))
    fv1_mu2_ = (V_opt(w, F, K, vol, T, alpha, mu=0.001)[0] -2* V_opt(w, F, K, vol, T, alpha, mu=0.0)[0] + V_opt(w, F, K, vol, T, alpha, mu=-0.001)[0])/(0.001**2)
    print('fv1_mu2={} fv1_mu2_={} diff={}'.format(fv1_mu2, fv1_mu2_, fv1_mu2-fv1_mu2_))
    fv1_mu3_ = (V_opt(w, F, K, vol, T, alpha, mu=0.001)[2] - V_opt(w, F, K, vol, T, alpha, mu=-0.001)[2])/0.002
    print('fv1_mu3={} fv1_mu3_={} diff={}'.format(fv1_mu3, fv1_mu3_, fv1_mu3-fv1_mu3_))

fv1=0.00353873457431449 fv2=0.003538734524837571 diff=4.9476919089519455e-11
fv1_mu=0.011769378713964494 fv1_mu_=0.01176938502568236 diff=-6.311717865373212e-09
fv1_mu2=0.029170195758220753 fv1_mu2_=0.02917019202054727 diff=3.7376734832172964e-09
fv1_mu3=0.03787032087727568 fv1_mu3_=0.037870292016371765 diff=2.8860903916738678e-08
fv1=0.0035387345577300314 fv2=0.003538734524837571 diff=3.2892460456734884e-11
fv1_mu=-0.008230633223956507 fv1_mu_=-0.008230630245545335 diff=-2.9784111722080953e-09
fv1_mu2=0.009170110269684633 fv1_mu2_=0.009170104870327017 diff=5.399357615523681e-09
fv1_mu3=0.01787047313625746 fv1_mu3_=0.017870440951417432 diff=3.218484002920352e-08


In [175]:
from scipy.optimize import brentq, newton, root_scalar
def solve_for_mu(F, vol, T, alpha, start_x=1.0,order=2,logstyle=False):
    def obj_fn(mu):
        fx,f1x,f2x,f3x = V_opt(CALL, F, F, vol, T, alpha, mu)
        gx,g1x,g2x,g3x = V_opt(PUT, F, F, vol, T, alpha, mu)
        return fx-gx,f1x-g1x,f2x-g2x,f3x-g3x
    def obj_fn_2(mu):
        fx,f1x,f2x,f3x = V_opt(CALL, F, F, vol, T, alpha, mu)
        gx,g1x,g2x,g3x = V_opt(PUT, F, F, vol, T, alpha, mu)
        vx,v1x,v2x,v3x = fx-gx+F,f1x-g1x,f2x-g2x,f3x-g3x
        
        hx = np.log1p((fx-gx)/F)
        h1x = v1x/vx
        h2x = v2x/vx - h1x**2
        h3x = v3x/vx + 2*(h1x**3)-3*h1x*v2x/vx
        return hx,h1x,h2x,h3x
    x  = start_x
    fn = obj_fn if not logstyle else obj_fn_2
    fx,f1x,f2x,f3x = fn(x)
    print('x={} fx={} f1x={}'.format(x,fx,f1x))
    step   = 1.0
    while abs(step)>1.0e-10:
        h = -fx/f1x
        if order == 1 or abs(fx)<1.0e-8:
            step = h
        elif order == 2:
            step = h/(1.0+0.5*h*f2x/f1x)
        elif order == 3:
            step = h*(1.0+0.5*f2x/f1x*h)/(1.0+f2x/f1x*h+(f3x/f1x)/6*h*h)
        else:
            assert False
        x += step
        fx,f1x,f2x,f3x = fn(x)
        print('x={} fx={} f1x={}'.format(x,fx,f1x))
    return x

In [178]:
alpha=0.15
solve_for_mu(F, vol, T, alpha, start_x=1.0,order=3, logstyle=True)

x=1.0 fx=1.0000011051921072 f1x=1.0000028960229597
x=-2.763182002829545e-05 fx=-2.7625673254222765e-05 f1x=1.0000010086349216
x=-6.174640128675692e-09 fx=4.119968255444916e-16 f1x=1.0000010087699596
x=-6.174640540672102e-09 fx=-2.168404344971009e-16 f1x=1.0000010087699598


-6.174640540672102e-09